# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,21.60,63,64,8.73,PN,1714930823
1,1,badr hunayn,23.7829,38.7905,28.51,32,0,1.52,SA,1714930922
2,2,cabo san lucas,22.8909,-109.9124,30.90,46,0,1.79,MX,1714930370
3,3,talnakh,69.4865,88.3972,-13.22,77,29,1.81,RU,1714930928
4,4,port-aux-francais,-49.3500,70.2167,3.77,89,87,3.41,TF,1714930930


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:


# Configure the map plot
city_weather_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    size = "Humidity")

# Display the map
city_weather_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df =  city_data_df.loc[(city_data_df["Wind Speed"]<5) & (city_data_df["Cloudiness"]==0) & (city_data_df["Max Temp"]<=27) & (city_data_df["Max Temp"]>=21)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
86,86,toliara,-23.3500,43.6667,22.67,77,0,1.78,MG,1714931107
215,215,saint-pierre,-21.3393,55.4781,22.82,64,0,3.60,RE,1714931389
233,233,sogndal,61.2283,7.1003,21.21,39,0,1.03,NO,1714931429
248,248,rakops,-21.0167,24.3333,22.92,31,0,3.01,BW,1714931463
261,261,diagonal mar,41.4052,2.2134,23.66,77,0,3.09,ES,1714931492
357,357,safaga,26.7292,33.9365,26.68,44,0,2.25,EG,1714931719
546,546,waddan,29.1614,16.1390,26.34,21,0,4.46,LY,1714932131


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,63,
1,badr hunayn,SA,23.7829,38.7905,32,
2,cabo san lucas,MX,22.8909,-109.9124,46,
3,talnakh,RU,69.4865,88.3972,77,
4,port-aux-francais,TF,-49.3500,70.2167,89,
...,...,...,...,...,...,...
572,toropets,RU,56.4978,31.6353,69,
573,constantia,RO,44.1833,28.6500,62,
574,tazacorte,ES,28.6290,-17.9293,76,
575,vilyuysk,RU,63.7553,121.6247,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "filter": "",
    "bias": "",
    "apiKey": geoapify_key,
    "limit": 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(url, params=params).json()
    # Convert the API response to JSON format
    # name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
badr hunayn - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
talnakh - nearest hotel: Талнах
port-aux-francais - nearest hotel: Keravel
vernon - nearest hotel: Okanagan Royal Park Inn
krutinka - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
waingapu - nearest hotel: HOTEL SANDLEWOOD
edinburgh of the seven seas - nearest hotel: No hotel found
ketchikan - nearest hotel: Cape Fox Lodge
hithadhoo - nearest hotel: Pebbles Inn
saudarkrokur - nearest hotel: Mikligarður
blackmans bay - nearest hotel: Villa Howden
utrik - nearest hotel: No hotel found
porto novo - nearest hotel: Hôtel 6500 CFA
badger - nearest hotel: Clarion Hotel & Suites
agat village - nearest hotel: Navy Getway Inns & Suites
iskateley - nearest hotel: Авантаж
kapuskasing - nearest hotel: Travelodge
port mathurin - nearest hotel: Escale Vacances
susuman - nearest hotel: No hotel found
kapa'a - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,63,No hotel found
1,badr hunayn,SA,23.7829,38.7905,32,No hotel found
2,cabo san lucas,MX,22.8909,-109.9124,46,Comfort Rooms
3,talnakh,RU,69.4865,88.3972,77,Талнах
4,port-aux-francais,TF,-49.3500,70.2167,89,Keravel
...,...,...,...,...,...,...
572,toropets,RU,56.4978,31.6353,69,Торопа
573,constantia,RO,44.1833,28.6500,62,Hotel Chérica
574,tazacorte,ES,28.6290,-17.9293,76,App Leyma
575,vilyuysk,RU,63.7553,121.6247,85,Мария


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    size = "Humidity",
    hover_cols=['Hotel Name', 'Country'])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)